In [ ]:
!pip install openai

In [ ]:
import os
import openai
import sys
from io import StringIO

In [ ]:
#Função para gerar o código usando o ChatGPT
def generate_code(question, error_message="", code =""):
    global rodada
    global first_question
    global deu_certo

    if deu_certo:
      return

    question += "\nVocê fará testes para validar o código e usará print() para printar o retorno. NÃO crie uma janela de visualização do retorno."

    if error_message != "":
      question += "\n\nDeu o seguinte erro e quero que conserte, me retornando o código completo corrigido Caso encontre outros erros conserte-os também." + error_message
      question += "\n\nEste é o código: \n" + code
    messages.append({"role": "user", "content": question})

    #Chama a API do OpenAI para gerar um código
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
    )

    chat_response = completion.choices[0].message.content

    #Divide o texto pelos delimitadores '```'
    elementos = chat_response.split('```')

    if (len(elementos) == 1):
      question = first_question
      if (error_message != ""):
        question += f"\nMe dê apenas o código completo e corrigido.\n\nErro: \n{error_message}\n\nCódigo: {code}"
      generate_code(question)

    #Obtém apenas os elementos que correspondem a trechos de código
    trechos_codigo = elementos[1::2]
    for i, codigo in enumerate(trechos_codigo):
      palavras = codigo.split(maxsplit=1)
      if palavras[0] == "python":
          trechos_codigo[i] = palavras[1]
    new_code = ""
    #Junta os trechos de código em uma única string, adicionando uma nova linha antes de cada junção
    new_code = new_code.join(['\n' + trecho if i > 0 else trecho for i, trecho in enumerate(trechos_codigo)])
    #new_code = new_code[7:]

    print("---------------------------------------------------------------------")
    print(f"Iteração: {rodada}\n\nCódigo gerado: \n\n{new_code}")
    print("---------------------------------------------------------------------")
    try:
        #Redirecionar a saída padrão para uma variável
        stdout = sys.stdout
        sys.stdout = StringIO()
        #Tenta executar o código gerado
        exec("" + new_code)
    except Exception as e:
        #Se houver um erro, imprime o erro e tenta gerar outro código
        #Restaurar a saída padrão
        sys.stdout = stdout
        print("---------------------------------------------------------------------")
        print(f"Erro do Código:\n\n {e}")
        print("---------------------------------------------------------------------")
        rodada += 1
        generate_code(first_question, error_message=f"{e}\n", code=new_code)
    else:
        #Capturar a saída impressa
        output = sys.stdout.getvalue()
        sys.stdout = stdout
        print("---------------------------------------------------------------------")
        print(f"Retorno do código: \n\n{output}")
        print("---------------------------------------------------------------------")
        message = f"Verifique se este código gerado faz exatamente o que foi exigido e se funciona direito. Responda extritamente apenas 'sim' ou 'nao'. Pergunta: {first_question}\n Código: {new_code}\n Retorno do código: {output}"
        #Se não houver erro, verifica se a resposta bate com a pergunta original
        messages.append({"role": "user", "content": message})
        completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=messages
        )

        chat_response = completion.choices[0].message.content
        resposta = chat_response[:3]
        print("---------------------------------------------------------------------")
        print(f"O código funciona? {resposta}")
        print("---------------------------------------------------------------------")
        if resposta.lower() == 'sim':
            #return new_code
            deu_certo = True
            print("\n\n---------------------------------------------------------------------")
            print(f"Codigo Final: \n\n{new_code}")
            print("---------------------------------------------------------------------")
        else:
            rodada += 1
            generate_code(first_question)



        #Se não houver erro, pede a confirmação do usuário
        print("O código respeita todas as exigências desse pedido?  [Sim, Não]")
        response = input(f"Pergunta: {first_question}\n")
        response = response.split(maxsplit=1)
        response = response[0]
        if response.lower() in ["sim", "s"]:
            #Se o usuário confirmar, atribui o novo código a `code` e retorna
            rodada = 1
            return True
        else:
            #Se o usuário negar, tenta gerar outro código
            rodada += 1
            generate_code(first_question)

Pedido de teste:
  Preciso de um código relativamente complexo em python com diversos erros de sintaxe para testar meu programa. Pode gerar um?

In [ ]:
print("----------------------------------------------")
print("BEM VINDO AO AUTO DEBBUGER AI.\nFAÇA UM PEDIDO E RETORNAREMOS UM CÓDIGO TOTALMENTE FUNCIONAL.")
print("----------------------------------------------\n\n")

opcao = input("1 - Fazer um pedido\n2 - Sair\n\n")
opcao = int(opcao)
while opcao != 2:
  if opcao != 1:
    print("Opção inválida. Tente novamente.\n")
  else:

      #Configurações para a chamada da API
      openai.api_key = "YOUR API KEY"
      messages = [
          {"role": "system", "content": "You're a python code generator, cold and to the point."}
      ]

      first_question = input("Seu pedido: ")
      question = ""
      question += first_question

      rodada = 1
      resposta = ""
      deu_certo = False

      #Chama a função 'generate_code()' e passa 'error_message' como um argumento opcional
      code = generate_code(question)
  opcao = input("\n\n\n1 - Fazer um pedido\n2 - Sair\n\n")
  opcao = int(opcao)

print("\n\nOBRIGADO POR USAR ESTE PROGRAMA.")